# Setting up Your Analytics Stack with Jupyter Notebook & AWS Redshift

In this blog post I will walk you though the exact steps needed to set up Jupyter Notebook to connect to your private data warehouse in AWS Redshift.

At RJMetrics, I use the Jupyter + Redshift analytics stack every single day because there are truly very few limits to what one can do with this powerful combination. From querying your data and visualizing it all in one place, to documenting your work and building interactive charts and dashboards, to running machine learning algorithms on top of your data and sharing the results with your team, the possibilities are limitless.

This entire post was written in Jupyter itself so that you can:
<ol>
    <li>see how easy it is to get started,
    <li><a href="https://github.com/3mei/test/tree/master/jupyter_redshift_analytics_stack">download the notebook</a> to follow along on your computer, and 
    <li>modify the code so that you can start analyzing your own data.
</ol>

Let's get started!

## What is Jupyter Notebook?

<img src='assets/jupyter.png'>

<a href="https://jupyter.org/">Jupyter Notebook</a> is an **open-source** data science tool used by many data scientists and data analysts at some of the most data-driven organizations in the world, including Google, Microsoft, IBM, Bloomberg, O'Reilly and NASA.

An extension of the <a href="http://ipython.org/">IPython project</a>, Jupyter Notebook is an application that runs directly in your browser and allows you to create and share documents with live code from over 40 different languages. You can also supplement these documents with explanatory text, equations, and rich visualizations.

Working with Jupyter is literally like writing a notebook. You can document your work, make it accessible to other people and easily reproduce it in the future. Once the notebook is set up, you can readily share it with others. For example, GitHub <a href="https://github.com/blog/1995-github-jupyter-notebooks-3">natively renders</a> notebook files as soon as they are committed into a repository. Alternatively, you can export your notebook into a number of different formats, including PDF, HTML, Markdown, and reST.

Combined with its open-source nature, big data integration capabilities (e.g., Apache Spark via Python, R, or Scala), and the ability to execute code in virtually any language of your choice, Jupyter Notebook is particularly well-suited for exploring data, experimenting with it, and capturing entire data-driven workflows (see also our <a href="https://blog.rjmetrics.com/2015/12/06/data-science-roundup-11-data-ethics-the-field-guide-to-data-science-and-solving-the-data-scientist-shortage/">Data Science Roundup #11</a> discussing the recent rise of notebooks).

#### <a id='installing_jupyter'></a>Installing Jupyter and Python

If you have never worked with Jupyter before or if you are not sure whether it is installed on your machine, please take a look at this brief <a href="http://jupyter.readthedocs.org/en/latest/install.html">installation article</a> from the developers of Jupyter. Several Python distributions (e.g., Anaconda) conveniently ship with both Python and Jupyter.

## What is AWS Redshift?

<a href="https://aws.amazon.com/redshift/">Amazon Redshift</a> is a fast, fully managed, petabyte-scale data warehouse. With Redshift, it has never been easier or more cost-effective to securely store your data in the cloud. On-demand pricing means that you can get started for as little as 25 cents an hour and you don't have to worry about backup, durability, availability, security, monitoring, or maintenance: AWS takes care of all of that for you.

Because of its ease of use, Redshift is experiencing <a href="https://blog.rjmetrics.com/2015/10/29/how-aws-redshift-is-opening-up-big-data-to-growth-hackers-marketers-and-other-light-quants/">rapid adoption</a> among light quants, growth hackers, seasoned analysts and data scientists. It integrates seamlessly with the languages you may be already familiar with, including SQL, R, and Python. For the purpose of this post, I will be focusing on the latter.

#### Setting Up Your AWS Redshift Instance

Setting up your Redshift instance is easy. AWS provides an excellent <a href="http://docs.aws.amazon.com/redshift/latest/gsg/getting-started.html">Getting Started with Amazon Redshift</a> guide and the whole process takes only a few minutes.

Once you have your Redshift instance up and running, I would encourage you to check out <a href="https://rjmetrics.com/product/pipeline">RJMetrics Pipeline</a>. Pipeline will connect to all of the disparate data sources you might have, including <a href="https://rjmetrics.com/integrations">databases and SaaS products</a>, and stream your data into one single place in Redshift. With Pipeline, you can start analyzing **all** of your data literally in minutes and not pay a penny if you stream less than 5 million rows/events. Here is a <a href="https://rjmetrics.wistia.com/medias/6tti03m4hm">four-minute tutorial</a> that will walk you through setting up your data warehouse to work with Pipeline.

## 1. Install Requirements

Please make sure Jupyter and Python are setup on your computer (see <a href='#installing_jupyter'>Installing Jupyter and Python</a> above).

We will need to install a few packages in order to connect to and work with AWS Redshift. Note that depending on your Python distribution, you may need to install additional packages. I recommend that you use the completely free <a href="https://www.continuum.io/why-anaconda">Anaconda distribution</a> as it already comes with many of the required packages installed and the packages are tested and guaranteed to work together. If working with Anaconda, you can use the built-in package management system `conda`. Alternatively, you can use Python's built-in <a href="https://pip.pypa.io/en/stable/installing/">pip</a> package management system or <a href="http://brew.sh/">Homebrew</a> (OS X only).

I will use a combination of Homebrew and pip in the following. Note that you may need to use pip3 if working with Python versions 3.3 or later.

<ol>
    <li>
        <b>Install PostgreSQL</b>
        <br>First, since Amazon Redshift is based on PostgreSQL 8.0.2, we will need a PostgreSQL client library.<br><br>
        If using Mac OS X, simply simply open up your terminal and type &nbsp;&nbsp; `brew install postgresql`<br><br>
        For other operating systems, please see the installation instructions <a href="http://www.postgresql.org/download/">here</a>.<br><br>
    </li>
    
    <li>
        <b>Install Python Requirements</b>
        <br>Second, we will need a few python packages to connect to and get data from Redshift.
        
        
        You can simply download the `requirements.txt` file, navigate to the folder containing it and type &nbsp;&nbsp; `pip install -r redshift_requirements.txt`.<br><br>
        
        (Pro tip: you may want to create a virtual python environment to keep the package dependencies required by different projects in separate locations. See <a href="http://docs.python-guide.org/en/latest/dev/virtualenvs/">Python</a> and <a href="http://conda.pydata.org/docs/_downloads/conda-pip-virtualenv-translator.html">Conda</a> documentation for more details.)<br><br>
        
        
        Here is a brief description of the four packages included in `redshift_requirements.txt`:
        <ul>
            <li><a href="http://initd.org/psycopg/">Psycopg2</a>: Python adapter needed to connect to Redshift.</li>

            <li><a href="http://www.sqlalchemy.org/">SQLAlchemy</a>: Python SQL toolkit and Object Relational Mapper that provides an efficient way to access a database.</li>
        
            <li><a href="https://github.com/catherinedevlin/ipython-sql">iPython-SQL</a>: provides a straightforward way to write SQL and get data back. Introduces magic commands `%sql` and `%%sql` so that you can write plain SQL and get back your query results in the form of a dataframe.</li>

            <li><a href="https://pypi.python.org/pypi/simplejson/">SimpleJSON</a>: simple and fast JSON encoder and decoder. We will store Redshift credentials in a JSON-formatted file and will make use of the package to read that file.</li>
       </ul>
   </li>
</ol>

## 2. Save your Redshift Credentials

To access you Redshift instance, you will need to know your (i) user name, (ii) password, (iii) host name, (iv) port and (v) database name. If you personally set up your Redshift instance, you will already know what these are. If not, please ask someone who did or take a look at the <a href="http://docs.aws.amazon.com/redshift/latest/mgmt/connecting-via-client-tools.html">AWS documentation</a> on how to obtain these credentials. Note that the port number for AWS Redshift is 5439.

So that you don't expose any of the sensitive information above in your Notebook (e.g, when sharing your notebook with other people or when storing your notebook in a GitHub repository), please save your Redshift credentials inside a JSON-formatted file called `redshift_creds.json.nogit`. You can download the template for that file <a href="https://github.com/3mei/rjmetrics_blog_posts/tree/master/jupyter_redshift_analytics_stack/assets/redshift_creds.json.nogit">here</a>. The contents of the file is pretty self-explanatory:


<img src='assets/redshift_creds.png'>

I would suggest saving the `redshift_creds.json.nogit` file in a folder that is separate from the one with your Jupyter file in it so that you don't accidentally store the credentials on GitHub. For extra security, you can specify the `*.nogit` pattern in the <a href="http://git-scm.com/docs/gitignore">.gitingore file</a> of your repository.

## 3. Whitelist your IP address in the AWS Panel

By default, your Redshift cluster is locked down so that nobody has access to it. You can grant people **inbound access** to your cluster via the AWS panel by creating a security group and specifying which IP addresses are allowed to connect to Redshift. Makes sure that your IP address is included in one of these rules. Please see the <a href="http://docs.aws.amazon.com/redshift/latest/mgmt/working-with-security-groups.html">Amazon Redshift Cluster Security Groups</a> documentation from AWS for more details.

## 4. Connect to Redshift!

You are now all set to connect to your Redshift instance! To do so from Jupyter, open up your terminal and type<br> `ipython notebook`

This command will start up a notebook server in your console and open up a web browser to show you the Jupyter Notebook dashboard:


<img src='assets/jupyter_dashboard.png'>

You can then navigate to the folder containing this notebook (get it from GitHub <a href="https://github.com/3mei/rjmetrics_blog_posts/tree/master/jupyter_redshift_analytics_stack">here</a>) and click on the "jupyter_redshift_analytics_stack.ipynb" file to start up the exact notebook you are looking at right now.

Let's import the packages we just installed and load SQL magic. You can press `(shift + enter)` to execute the cell below

In [1]:
import sqlalchemy
import psycopg2
import simplejson
%load_ext sql
%config SqlMagic.displaylimit = 5

<br>Next, let's read the credentials from file. Note that you will need to specify the path to the `redshift_creds.json.nogit` file on your machine.

In [2]:
with open("/PATH/TO/YOUR/redshift_creds.json.nogit") as fh:
    creds = simplejson.loads(fh.read())

<br>Finally, let's connect to Redshift!

In [3]:
connect_to_db = 'postgresql+psycopg2://' + \
                creds['user_name'] + ':' + creds['password'] + '@' + \
                creds['host_name'] + ':' + creds['port_num'] + '/' + creds['db_name'];
%sql $connect_to_db

u'Connected: user_name@db_name'

<br>
As a simple test, let's get the current date and time from our Redshift instance:

In [4]:
%%sql
select getdate() as current_date

1 rows affected.


current_date
2016-02-08 15:30:49


We can now store the result in a dataframe by simply typing

In [5]:
df = _.DataFrame()
df

,current_date
0,2016-02-08 15:30:49


That's it!

With only a few commands you loaded your Redshift credentials, connected to your data warehouse, executed a query and converted the query result into a pandas dataframe! Now the world is your oyster: you can use any package in the Python ecosystem to slice and dice a dataframe, visualize your results and share them with others. 

More on this in the blog posts to follow.